In [ ]:
import json
import copy
import spacy
from spacy.lang.en import English
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer

import os, sys
import json
import sqlite3
import traceback
import argparse
import string
from tqdm.notebook import tqdm

# from spider.process_sql import tokenize, get_schema, get_tables_with_alias, Schema, get_sql
from spider import process_sql, evaluation
from SpeakQL.Allennlp_models.utils import misc_utils
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, EditDistance

import numpy as np
from copy import copy, deepcopy
from collections import defaultdict, Counter

import importlib

In [ ]:
importlib.reload(process_sql)
importlib.reload(evaluation)
importlib.reload(misc_utils)

### Evaluate

In [ ]:
db = 'concert_singer'
g_str = "SELECT count(*) FROM singer WHERE singer.name = 'Joe Sharp'"
p_str = "SELECT Count(*) FROM singer WHERE singer.Name = 'terminal'"

db, p_str, g_str, EvaluateSQL(p_str, g_str, db)

### Global file paths

In [ ]:
split_index_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_dev.txt'
split_index_test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_test.txt'

### Collect questions (input to Polly)

In [ ]:
train_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/train.json'
dev_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
with open(train_json_path, 'r') as f:
    train_dataset = json.load(f)
with open(dev_json_path, 'r') as f:
    dev_dataset = json.load(f)

len(train_dataset), len(dev_dataset)

In [ ]:
# TODO: +train_others.json?

### Dev/test split
- Create an index file for official dev set, showing which samples are in dev/test
- Do not rerun, since we want to keep the split the same across methods so as to compare fairly

In [ ]:
# Collect & split databases 
official_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
with open(official_dev_path, 'r') as f:
    offc_dev_dataset = json.load(f)

offc_dev_dbs = Counter([d['db_id'] for d in offc_dev_dataset])
offc_dev_dbs.most_common()

In [ ]:
speakql_dev_dbs = []
speakql_test_dbs = []

for i, pair in enumerate(offc_dev_dbs.most_common()):
    db_id, cnt = pair
    if i % 2 == 1:
        speakql_dev_dbs.append(db_id)
    else:
        speakql_test_dbs.append(db_id)

speakql_dev_dbs, speakql_test_dbs

In [ ]:
speakql_dev_ids = []
speakql_test_ids = []

for i, d in enumerate(offc_dev_dataset):
    if d['db_id'] in speakql_dev_dbs:
        speakql_dev_ids.append(i)
    else:
        speakql_test_ids.append(i)

len(speakql_dev_ids), len(speakql_test_ids)

In [ ]:
# Uncomment to (over)write these results 

# with open(split_index_dev_path, 'w') as f:
#     for i in speakql_dev_ids:
#         f.write('{}\n'.format(i))

# with open(split_index_test_path, 'w') as f:
#     for i in speakql_test_ids:
#         f.write('{}\n'.format(i))

### Build NLIDB input
- Did not modify from IRNet code; the purpose of this part is to generate '{train|dev}_asr_amazon.json', which are dataset files with transcriptions.
- The only difference: IRNet needs to skip certain instances, here we don't need to skip
- As long as 'train/dev_asr_amazon.json' already exists, we don't have to run this again

In [ ]:
DATASET = 'train'

dataset_train_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/train_spider.json'
dataset_dev_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
dataset_json_path = dataset_train_json_path if DATASET == 'train' else dataset_dev_json_path

trans_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/spider-{0}-batch0/'.format(DATASET)

dataset_asr_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon.json'.format(DATASET)


In [ ]:
# Paths for human test 

# DATASET = 'human-test-yshao'

# dataset_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

# trans_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/speech/yshao_asr'

# dataset_asr_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon.json'


In [ ]:
with open(dataset_json_path, 'r') as f:
    dataset = json.load(f)
len(dataset)

In [ ]:
dataset[0].keys()

In [ ]:
dataset_asr = []

for i, d in tqdm(enumerate(dataset)):
    # trans_json_fname = os.path.join(trans_dir, 'Transcription-{}-batch0-{}.wav.json'.format(DATASET, i))
    trans_json_fname = os.path.join(trans_dir, 'Transcription-{}-{}.wav.json'.format(DATASET, i))
    
    trans = json.load(open(trans_json_fname, 'r'))
    for alter in trans['results']['segments'][0]['alternatives']:
        tokens = []
        span_ranges = []
        for item in alter['items']:
            token = item['content']
            if item['type'] == 'punctuation':
                st = ed = 0
            else:
                st = item['start_time']
                ed = item['end_time']
            tokens.append(token)
            span_ranges.append((st, ed))

        d_asr = deepcopy(d)
        d_asr['question'] = alter['transcript']
        d_asr['question_toks'] = tokens
        d_asr['span_ranges'] = span_ranges
        d_asr['original_id'] = i
        
        dataset_asr.append(d_asr)

len(dataset_asr), dataset_asr[0].keys()

In [ ]:
# Uncomment to (over)write these results 

# with open(dataset_asr_json_path, 'w') as f:
#     json.dump(dataset_asr, f, indent=4)

### Build Reranker input
- The dataset is a list of list
    - Outer list: original samples in Spider
    - Inner list: transcription candidates for an original sample
- Add evaluated score of ratsql predicted SQL (partial match average)
- Group samples by original_id
- Split dev/test based on given index files

In [ ]:
DATASET = 'train' # 'train', 'dev' 

dataset_in_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{}/{}_asr_amazon_RatsqlPredicted.json'.format(DATASET, DATASET)
# gold_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/IRNet/output/asr_amazon/{}/gold_full.txt'.format(DATASET)
# pred_sql_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/IRNet/output/asr_amazon/{}/output_full.txt'.format(DATASET)

original_train_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/train_spider.json'
original_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
original_dataset_path = original_train_path if DATASET == 'train' else original_dev_path

reranker_train_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json'
reranker_dev_full_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_reranker(full).json'
reranker_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_reranker.json'
reranker_test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'


In [ ]:
# Paths for human test 

# DATASET = 'human-test-yshao'

# dataset_in_json_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon_RatsqlPredicted.json'

# original_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

# test_reranker_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'


In [ ]:
with open(dataset_in_json_path, 'r') as f:
    dataset_in = json.load(f)
# with open(gold_path, 'r') as f:
#     gold_list = [l.strip() for l in f.readlines()]
# with open(pred_sql_path, 'r') as f:
#     pred_sql_list = [l.strip() for l in f.readlines()]

with open(original_dataset_path, 'r') as f:
    original_samples = json.load(f)
    
len(dataset_in), len(original_samples)

In [ ]:
dataset_in[500].keys()

In [ ]:
# Construct reranker dataset samples 

dataset_reranker = defaultdict(list)

for i in tqdm(range(len(dataset_in))):

    d = deepcopy(dataset_in[i])
    o_id = d['original_id']
    o_sample = original_samples[o_id]
    assert ' '.join(d['query'].split()) == ' '.join(o_sample['query'].split()), \
        '{}:\n{}\n{}'.format(i, ' '.join(d['query'].split()), ' '.join(o_sample['query'].split()))
    
    d['gold_question'] = o_sample['question']
    d['gold_question_toks'] = o_sample['question_toks']
    
    p_str = d['ratsql_pred_sql']
    g_str = d['query']
    db = d['db_id']
    
    exact, partial_score = EvaluateSQL(p_str, g_str, db)
    d['ratsql_pred_exact'] = exact
    d['ratsql_pred_score'] = partial_score
    
    e_dist, _ = EditDistance(d['question_toks'], d['gold_question_toks'])
    d['question_toks_edit_distance'] = int(e_dist)
    
    dataset_reranker[o_id].append(d)
    
len(dataset_reranker[80])

In [ ]:
d

In [ ]:
for cand in dataset_reranker[80]:
    print('Question (transcription): {}'.format(cand['question']))
    print('Question (gold): {}'.format(cand['gold_question']))
    print('Gold: {}'.format(cand['query']))
    print('Pred: {}'.format(cand['ratsql_pred_sql']))
    print('Score: {}'.format(cand['ratsql_pred_score']))
    print('Exact: {}'.format(cand['ratsql_pred_exact']))
    print('Edit Dist: {}'.format(cand['question_toks_edit_distance']))
    print('='*50)

In [ ]:
len(dataset_reranker)

In [ ]:
dataset_reranker_list = []

N = max(dataset_reranker.keys())
for o_id in range(N + 1):
    dataset_reranker_list.append(dataset_reranker[o_id])
len(dataset_reranker_list)

In [ ]:
if DATASET == 'train':
    with open(reranker_train_path, 'w') as f:
        json.dump(dataset_reranker_list, f, indent=4)

In [ ]:
if DATASET == 'dev':
    with open(split_index_dev_path, 'r') as f:
        dev_ids = [int(l) for l in f.readlines()]
    with open(split_index_test_path, 'r') as f:
        test_ids = [int(l) for l in f.readlines()]
    
    dev_list = [dataset_reranker_list[i] for i in dev_ids]
    test_list = [dataset_reranker_list[i] for i in test_ids]
    assert len(dev_list) + len(test_list) == len(dataset_reranker_list)
    
    with open(reranker_dev_path, 'w') as f:
        json.dump(dev_list, f, indent=4)
    with open(reranker_test_path, 'w') as f:
        json.dump(test_list, f, indent=4) 
    with open(reranker_dev_full_path, 'w') as f:
        json.dump(dataset_reranker_list, f, indent=4) 

In [ ]:
if DATASET.startswith('human-test'):
    with open(test_reranker_path, 'w') as f:
        json.dump(dataset_reranker_list, f, indent=4)

### Fast Aligner

In [ ]:
# Load samples, write parallel sentences to file 

DATASET = 'human-test-yshao' # 'train-dev' or 'human-test-*'

reranker_train_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json'
spider_train_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/train_spider.json'

reranker_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_reranker(full).json'
spider_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

reranker_human_test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'
spider_human_test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

parallel_txt_path = 'alignments/human_test/parallel.txt'
# parallel_txt_train_path = 'alignments/train_set_only/parallel_train.txt' ## Fully unsupervised aligner, no training and test 

In [ ]:
with open(reranker_train_path, 'r') as f:
    reranker_train_samples = json.load(f)
with open(spider_train_path, 'r') as f:
    spider_train_samples = json.load(f)
with open(reranker_dev_path, 'r') as f:
    reranker_dev_samples = json.load(f)
with open(spider_dev_path, 'r') as f:
    spider_dev_samples = json.load(f)
    
len(reranker_train_samples), len(spider_train_samples), len(reranker_dev_samples), len(spider_dev_samples)

In [ ]:
with open(reranker_human_test_path, 'r') as f:
    reranker_human_test_samples = json.load(f)
with open(spider_human_test_path, 'r') as f:
    spider_human_test_samples = json.load(f)

len(reranker_human_test_samples), len(spider_human_test_samples)

In [ ]:
reranker_train_samples[0][0].keys()

In [ ]:
spider_train_samples[0].keys()

In [ ]:
train_pairs = []
for sample, orig_sample in zip(reranker_train_samples, spider_train_samples):
    for cand in sample:
        src_sen = ' '.join(cand['question_toks'])
        tgt_sen = ' '.join(orig_sample['question_toks'])
        train_pairs.append((src_sen, tgt_sen))
len(train_pairs)

In [ ]:
dev_pairs = []
for sample, orig_sample in zip(reranker_dev_samples, spider_dev_samples):
    for cand in sample:
        src_sen = ' '.join(cand['question_toks'])
        tgt_sen = ' '.join(orig_sample['question_toks'])
        dev_pairs.append((src_sen, tgt_sen))
len(dev_pairs)

In [ ]:
human_test_pairs = []
for sample, orig_sample in zip(reranker_human_test_samples, spider_human_test_samples):
    for cand in sample:
        src_sen = ' '.join(cand['question_toks'])
        tgt_sen = ' '.join(orig_sample['question_toks'])
        human_test_pairs.append((src_sen, tgt_sen))
len(human_test_pairs)

In [ ]:
human_test_pairs[::100]

In [ ]:
if DATASET.startswith('human-test'):
    sentence_pairs = human_test_pairs
elif DATASET == 'train-dev':
    sentence_pairs = train_pairs + dev_pairs
else:
    raise

In [ ]:
# Uncomment to (over)write these results 

# with open(parallel_txt_path, 'w') as f:
# #     for s, t in train_pairs:
# #         f.write(s + ' ||| ' + t + '\n')
# #     for s, t in dev_pairs:
# #         f.write(s + ' ||| ' + t + '\n')

#     for s, t in sentence_pairs:
#         f.write(s + ' ||| ' + t + '\n')

After running fast_align and atools, check & load the alignment 

In [ ]:
if DATASET == 'human-test-yshao':
    alignment_txt_path = 'alignments/human_test_yshao/symmetry.align'
elif DATASET == 'train-dev':
    alignment_txt_path = 'alignments/symmetry.align'
else:
    raise

with open(alignment_txt_path, 'r') as f:
    alignment_lines = f.read().strip().split('\n')
len(alignment_lines)

In [ ]:
def alignment_seq_to_dict(seq, src_str, tgt_str):
    # Translating an alignment seq (0-0, 1-2, 2-3, ...) into dicts 
    # Pairs are sorted by src_index 
    align_items = seq.split(' ')
    src_sen = src_str.split(' ')
    tgt_sen = tgt_str.split(' ')
    
    def _span_match(src_span : tuple,
                    tgt_span : tuple):
        src_span_str = ''.join(src_sen[src_span[0] : src_span[-1] + 1])
        tgt_span_str = ''.join(tgt_sen[tgt_span[0] : tgt_span[-1] + 1])
        edist, _ = EditDistance(src_span_str, tgt_span_str)
        match = max(len(src_span_str), len(tgt_span_str)) - edist
        return match
    
    forward_dict = defaultdict(list)
    backward_dict = defaultdict(list)
    for align_item in align_items:
        i_str, j_str = align_item.split('-')
        i = int(i_str)
        j = int(j_str)
        forward_dict[i].append(j)
        backward_dict[j].append(i)
    
    # Continuous: selecting spans 
    forward_span_dict = defaultdict(tuple)  # Reconstructed every iteration 
    backward_span_dict = defaultdict(tuple) # Reconstructed every iteration
    forward_dict_2 = copy(forward_dict)     # Maintained over iterations 
    backward_dict_2 = copy(backward_dict)   # Maintained over iterations 

#     print('-init-')
#     print(forward_dict_2)
#     print(backward_dict_2)
#     print(forward_span_dict)
#     print(backward_span_dict)
    
    modified = True
    
    while modified:
        modified = False
        forward_span_dict.clear()
        backward_span_dict.clear()
        
        for i in range(len(src_sen)):
            tgt_ids = forward_dict_2[i]
            if len(tgt_ids) == 0:
                continue

            spans = []
            cur_span = []
            for j in tgt_ids:
                if len(cur_span) == 0:
                    cur_span.append(j)
                elif j == cur_span[-1] + 1:
                    cur_span.append(j)
                else:
                    spans.append(tuple(cur_span))
                    cur_span.clear()
                    cur_span.append(j)
            if len(cur_span) > 0:
                spans.append(tuple(cur_span))
                cur_span.clear()
                
            # Selecting best span 
            if len(spans) == 1:
                best_span = spans[0]
            else:
                best_span = tuple()
                max_match = -np.inf
                for span in spans:
                    match = _span_match((i,), span)
                    if match > max_match:
                        best_span = span
                        max_match = match

            forward_span_dict[i] = best_span
            forward_dict_2[i] = list(best_span)
            for j in tgt_ids:
                if j not in best_span:
                    modified = True
                    backward_dict_2[j].remove(i)

#         print('-1-')
#         print(forward_dict_2)
#         print(backward_dict_2)
#         print(forward_span_dict)
#         print(backward_span_dict)

        # For backward, use the backward_dict_2 just updated  
        for j in range(len(tgt_sen)):
            src_ids = backward_dict_2[j]
            if len(src_ids) == 0:
                continue

            spans = []
            cur_span = []
            for i in src_ids:
                if len(cur_span) == 0:
                    cur_span.append(i)
                elif i == cur_span[-1] + 1:
                    cur_span.append(i)
                else:
                    spans.append(tuple(cur_span))
                    cur_span.clear()
                    cur_span.append(i)
            if len(cur_span) > 0:
                spans.append(tuple(cur_span))
                cur_span.clear()

            # Selecting best span 
            if len(spans) == 1:
                best_span = spans[0]
            else:
                best_span = tuple()
                max_match = -np.inf
                for span in spans:
                    match = _span_match(span, (j,))
                    if match > max_match:
                        best_span = span
                        max_match = match

            backward_span_dict[j] = best_span
            backward_dict_2[j] = list(best_span)
            for i in src_ids:
                if i not in best_span:
                    modified = True
                    forward_dict_2[i].remove(j)

#         print('-2-')
#         print(forward_dict_2)
#         print(backward_dict_2)
#         print(forward_span_dict)
#         print(backward_span_dict)
    
    # Merging overlapping spans 
    src_spans = [(i,) for i in range(len(src_sen))]   # [i]: span containing i 
    tgt_spans = [(j,) for j in range(len(tgt_sen))]   # [j]: span containing j 
    
    modified = True
    
    while modified:
        modified = False
        for i in range(len(src_sen)):
            span = src_spans[i]
            span_set = set(span)
            for j in forward_dict_2[i]:
                for _i in backward_dict_2[j]:
                    # merge _i into span with i 
                    if _i == i or src_spans[_i] == span:
                        continue
                    span_set.update(src_spans[_i])
                    modified = True
            
            new_span = tuple(sorted(span_set))
            if new_span != span:
                for _i in new_span:
                    src_spans[_i] = new_span
                    
        for j in range(len(tgt_sen)):
            span = tgt_spans[j]
            span_set = set(span)
            for i in backward_dict_2[j]:
                for _j in forward_dict_2[i]:
                    # merge _j into span with j 
                    if _j == j or tgt_spans[_j] == span:
                        continue
                    span_set.update(tgt_spans[_j])
                    modified = True
            
            new_span = tuple(sorted(span_set))
            if new_span != span:
                for _j in new_span:
                    tgt_spans[_j] = new_span
    
    # Reconstructing span dict (span to span)
    span_dict = defaultdict(tuple)
    
#     for _, src_span in backward_span_dict.items():
#         if len(src_span) == 0:
#             continue
        
#         tgt_span = forward_span_dict[src_span[0]]
#         assert len(tgt_span) > 0
#         for i in src_span:
#             assert forward_span_dict[i] == tgt_span, '{}\n{}'.format(forward_span_dict, backward_span_dict)
        
#         span_dict[src_span] = tgt_span

    for i in range(len(src_sen)):
        if len(forward_dict_2[i]) == 0:
            continue
            
        src_span = src_spans[i]
        
        if src_spans[i] in span_dict:
            continue
            
        tgt_span = tgt_spans[forward_dict_2[i][0]]
        span_dict[src_span] = tgt_span
    
    # Monotonic 
    last_src_span = None
    for src_span in sorted(span_dict.keys()):
        tgt_span = span_dict[src_span]
        
        if last_src_span is None:
            last_src_span = src_span
            continue
        
        last_tgt_span = span_dict[last_src_span]
        
        if tgt_span[0] > last_tgt_span[-1]:
            # no conflict 
            last_src_span = src_span
            continue
        
        # conflict 
        match = _span_match(src_span, tgt_span)
        last_match = _span_match(last_src_span, last_tgt_span)
        
        if match > last_match:
            del span_dict[last_src_span]
            last_src_span = src_span
        else:
            del span_dict[src_span]
    
    # N-gram pairs 
    pairs = []
    for src_span, tgt_span in sorted(span_dict.items()):
        src_str = ' '.join(src_sen[src_span[0] : src_span[-1] + 1])
        tgt_str = ' '.join(tgt_sen[tgt_span[0] : tgt_span[-1] + 1])
        pairs.append((src_str, tgt_str))
    
    return span_dict, pairs
    
    # TODO?: Forcing monotonic and continuous more wisely 
    

In [ ]:
idx = 180
alignment_lines[idx], human_test_pairs[idx][0], human_test_pairs[idx][1]

In [ ]:
alignment_seq_to_dict(alignment_lines[idx], human_test_pairs[idx][0], human_test_pairs[idx][1])

In [ ]:
alignment_results = []

for idx in tqdm(range(len(alignment_lines))):
    alignment_results.append(alignment_seq_to_dict(alignment_lines[idx], sentence_pairs[idx][0], sentence_pairs[idx][1]))

In [ ]:
alignment_results[540]

### Build Rewriter input
- Each sample correspond to a sample in original spider dataset; a list of candidates (like in reranker)
- Adding: alignment idx pairs, text span pairs, seq tags, list of rewrites (where edits are needed). Each rewrite has a text span, span st/ed idx, target text span, target st/ed idx (<- target info shouldn't be used in test)

In [ ]:
DATASET = 'train'   # as in original; 'train' for train, 'dev' for dev + test 

reranker_paths = {
    'train': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json',
    'dev': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_reranker.json',
    'test': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json',
    'dev-full': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_reranker(full).json'
}

original_train_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/train_spider.json'
original_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

rewriter_paths = {
    'train': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_rewriter.json',
    'dev': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_rewriter.json',
    'test': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json',
    'dev-full': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_rewriter(full).json'
}

split_index_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_dev.txt'
split_index_test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_test.txt'

In [ ]:
reranker_path = reranker_paths['train'] if DATASET == 'train' else reranker_paths['dev-full']
original_path = original_train_path if DATASET == 'train' else original_dev_path

In [ ]:
# Paths for human test 

# DATASET = 'human-test-yshao'

# reranker_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'

# original_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

# rewriter_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'


In [ ]:
with open(reranker_path, 'r') as f:
    reranker_samples = json.load(f)
with open(original_path, 'r') as f:
    original_samples = json.load(f)

len(reranker_samples), len(original_samples)

In [ ]:
rewriter_samples = []

if DATASET == 'train':
    global_cand_id = 0
elif DATASET == 'dev':
    global_cand_id = len(train_pairs)
else:
    # human test 
    global_cand_id = 0
    
global_question_pairs = sentence_pairs

for reranker_sample, original_sample in zip(reranker_samples, original_samples):
    rewriter_sample = copy(reranker_sample)
    for cand in rewriter_sample:
        assert ' '.join(cand['question_toks']) == global_question_pairs[global_cand_id][0], \
            '{}:\n{}\n{}'.format(global_cand_id, cand, global_question_pairs[global_cand_id])
        assert ' '.join(cand['gold_question_toks']) == ' '.join(original_sample['question_toks']) == global_question_pairs[global_cand_id][1], \
            '{}:\n{}\n{}'.format(global_cand_id, cand, global_question_pairs[global_cand_id])
        
        # Skip global_cand_id if skipped by IRNet 
        while (' '.join(cand['question_toks']), ' '.join(cand['gold_question_toks'])) != global_question_pairs[global_cand_id]:
            print('='*10, global_cand_id, '='*10)
            print(cand['question'], cand['gold_question'])
            print(global_question_pairs[global_cand_id])
            global_cand_id += 1
            if global_cand_id >= len(global_question_pairs):
                raise ValueError
        
        alignment_span_dict, alignment_text_pairs = alignment_results[global_cand_id]
        alignment_span_pairs = sorted(alignment_span_dict.items())
        cand['alignment_span_pairs'] = alignment_span_pairs
        cand['alignment_text_pairs'] = alignment_text_pairs
        
        tags = []
        edits = []
        last_src_idx = -1
        for span_pair, text_pair in zip(alignment_span_pairs, alignment_text_pairs):
            src_span, tgt_span = span_pair
            src_span_txt, tgt_span_txt = text_pair
            assert src_span[0] > last_src_idx
            if src_span[0] > last_src_idx + 1:
                del_span_len = src_span[0] - last_src_idx - 1
                if del_span_len == 1:
                    tags += ['U-DEL']
                else:
                    tags += ['B-DEL'] + ['I-DEL'] * (del_span_len - 2) + ['L-DEL']
            
            if text_pair[0].lower() == text_pair[1].lower():
                assert len(span_pair[0]) == len(span_pair[1])
                tags += ['O-KEEP'] * len(span_pair[0])
            else:
                # Edit needed 
                if len(src_span) == 1:
                    tags += ['U-EDIT']
                else:
                    tags += ['B-EDIT'] + ['I-EDIT'] * (len(src_span) - 2) + ['L-EDIT']
                edits.append({
                    'src_span': src_span,
                    'tgt_span': tgt_span,
                    'src_text': src_span_txt,
                    'tgt_text': tgt_span_txt
                })
            last_src_idx = src_span[-1]
            
        del_span_len = len(cand['question_toks']) - last_src_idx - 1
        if del_span_len == 0:
            pass
        elif del_span_len == 1:
            tags += ['U-DEL']
        else:
            tags += ['B-DEL'] + ['I-DEL'] * (del_span_len - 2) + ['L-DEL']
        
        assert len(tags) == len(cand['question_toks']), \
            '\n'.join([str(obj) for obj in (tags, cand['question_toks'], cand['gold_question_toks'], \
                                            alignment_text_pairs, alignment_span_pairs, edits)])
        cand['rewriter_tags'] = tags
        cand['rewriter_edits'] = edits
        
        global_cand_id += 1
    
    rewriter_samples.append(rewriter_sample)

len(rewriter_samples)

In [ ]:
rewriter_samples[5][1].keys()

In [ ]:
rewriter_samples[5][2]['question'], rewriter_samples[5][2]['gold_question'], \
rewriter_samples[5][2]['rewriter_tags'], rewriter_samples[5][2]['rewriter_edits']

In [ ]:
if DATASET == 'train':
    with open(rewriter_paths['train'], 'w') as f:
        json.dump(rewriter_samples, f, indent=4)

In [ ]:
if DATASET == 'dev':
    with open(split_index_dev_path, 'r') as f:
        dev_ids = [int(l) for l in f.readlines()]
    with open(split_index_test_path, 'r') as f:
        test_ids = [int(l) for l in f.readlines()]
    
    dev_list = [rewriter_samples[i] for i in dev_ids]
    test_list = [rewriter_samples[i] for i in test_ids]
    assert len(dev_list) + len(test_list) == len(rewriter_samples)
    
    with open(rewriter_paths['dev'], 'w') as f:
        json.dump(dev_list, f, indent=4)
    with open(rewriter_paths['test'], 'w') as f:
        json.dump(test_list, f, indent=4)
    with open(rewriter_paths['dev-full'], 'w') as f:
        json.dump(rewriter_samples, f, indent=4)

In [ ]:
if DATASET.startswith('human-test'):
    with open(rewriter_path, 'w') as f:
        json.dump(rewriter_samples, f, indent=4)

In [ ]:
# Check for edits 
for rewriter_sample in rewriter_samples:
    for cand in rewriter_sample:
        tags = cand['rewriter_tags']
        edits = cand['rewriter_edits']
        for edit in edits:
            if len(edit['tgt_span']) > 1 and 'B-EDIT' in cand['rewriter_tags']:
                print(cand['original_id'])
                print(cand['question'])
                print(cand['gold_question'])
                print(cand['rewriter_tags'])
                print(cand['rewriter_edits'])
                break


#### Build a 'fake' spider for BRIDGE

In [ ]:
spider_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider'
bridge_train_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/Bridge-train'
bridge_infer_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/Bridge-infer'
bridge_infer_asr_dir = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/Bridge-asr-infer'
dev_index_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_dev.txt'
test_index_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_test.txt'

## Train (orig) dir: train = train, original queries; dev = dev split, original queries 
## Infer (orig) dir: train = empty; dev = test split, original queries 
## Infer (asr) dir: train = empty; dev = test split, ASR queries (first cands) 

os.makedirs(bridge_train_dir, exist_ok=True)
os.makedirs(bridge_infer_dir, exist_ok=True)
os.makedirs(bridge_infer_asr_dir, exist_ok=True)

# in_train_json = os.path.join(spider_dir, 'my/train/train_reranker.json')
# in_dev_json = os.path.join(spider_dir, 'my/dev/dev_reranker.json')
# in_test_json = os.path.join(spider_dir, 'my/dev/test_reranker.json')

# out_train_json = os.path.join(bridge_train_dir, 'train.json')
# out_dev_json = os.path.join(bridge_train_dir, 'dev.json')
# out_test_json = os.path.join(bridge_infer_dir, 'dev.json')

In [ ]:
# with open(in_train_json, 'r') as f:
#     in_train = json.load(f)
# with open(in_dev_json, 'r') as f:
#     in_dev = json.load(f)
# with open(in_test_json, 'r') as f:
#     in_test = json.load(f)
    
# len(in_train), len(in_dev), len(in_test)

In [ ]:
## Orig dev/test 
with open(dev_index_path, 'r') as f:
    dev_indices = [int(l) for l in f.read().strip().split('\n')]
with open(test_index_path, 'r') as f:
    test_indices = [int(l) for l in f.read().strip().split('\n')]
with open(os.path.join(spider_dir, 'dev.json'), 'r') as f:
    orig_dev = json.load(f)
print(len(dev_indices), len(test_indices), len(orig_dev))

In [ ]:
# BRIDGE only reads 'train.json' and 'dev.json'

with open(os.path.join(bridge_train_dir, 'dev.json'), 'w') as f:
    json.dump([orig_dev[idx] for idx in dev_indices], f, indent=4)
    
with open(os.path.join(bridge_infer_dir, 'dev.json'), 'w') as f:
    json.dump([orig_dev[idx] for idx in test_indices], f, indent=4)

In [ ]:
## ASR test  
# with open(test_index_path, 'r') as f:
#     test_indices = [int(l) for l in f.read().strip().split('\n')]
with open(os.path.join(spider_dir, 'my/dev/test_reranker.json'), 'r') as f:
    reranker_test = json.load(f)

In [ ]:
with open(os.path.join(bridge_infer_asr_dir, 'dev.json'), 'w') as f:
    json.dump([d[0] for d in reranker_test], f, indent=4)

### Split according to indexes

In [ ]:
dev_full_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_rewriter(full)+phonemes.json'
dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_rewriter+phonemes.json'
test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter+phonemes.json'

In [ ]:
with open(dev_full_path, 'r') as f:
    dev_full_set = json.load(f)

with open(split_index_dev_path, 'r') as f:
    dev_ids = [int(l) for l in f]
with open(split_index_test_path, 'r') as f:
    test_ids = [int(l) for l in f]
dev_set = [dev_full_set[i] for i in dev_ids]
test_set = [dev_full_set[i] for i in test_ids]
assert len(dev_full_set) == len(dev_set) + len(test_set), (len(dev_full_set), len(dev_set), len(test_set))

with open(dev_path, 'w') as f:
    json.dump(dev_set, f, indent=2)
with open(test_path, 'w') as f:
    json.dump(test_set, f, indent=2)
    